In [44]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
import torch
import pyvene as pv
import pprint
from task import MetaLearningTask, FineTuningTask
from transformers import BatchEncoding, PretrainedConfig
import jax.random as jr
from torch2jax import j2t, t2j
import dataclasses
import pyreft
import nvidia_smi
from models.mamba import MambaLMHeadModel
from dataclasses import dataclass
from mamba_ssm.utils.generation import InferenceParams
from task import DynamicIntervenableModel, TuneConfig, LoraConfig, ReftConfig
import gc
from models.gpt import GPT, GPTConfig
from models.mamba import MambaLMHeadModel, MambaConfig
from pympler import asizeof
from data.hmm import CompositionalHMMDataset, CompositionalHMMDatasetConfig
import lightning as L
from torch.nn import init
from peft import PeftModel
import jax.numpy as jnp
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from functools import partial
import jax

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
IDS = {'mamba': 'ale6kz0k', 'gpt': 'duy6u49u'}

In [72]:
a = [1,2]

In [76]:
from typing import Iterable


isinstance(0, Iterable)

False

In [63]:
task = FineTuningTask(
    TuneConfig(
        IDS["gpt"],
        method_config=LoraConfig(r=4, target_modules="all-linear"),
        context_length=[50,50],
        constraints=[[1, 0], [2, 0], [4, 1], [6, 0], [7, 0]],
        precompute_pp=True,
        prefix_size=(20,30)
    )
).cuda()

number of parameters: 10.67M
Loaded dataset : (11288/1000)


Seed set to 42


Loaded checkpoing : last.ckpt
Precomputing fine-tuning oracle...


1000it [00:17, 57.22it/s]

Done


In [69]:
trainer = L.Trainer(
    max_steps=100,
    accelerator="gpu",
    enable_checkpointing=False,
    val_check_interval=50,
    reload_dataloaders_every_n_epochs=1,
    check_val_every_n_epoch=None,
)


/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mila/l/leo.gagnon/latent_control/venv/lib/pyth ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [71]:
trainer.fit(task)

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mila/l/leo.gagnon/latent_control/venv/lib/pyth ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params | Mode
-------------------------------------------
0 | model | PeftModel | 10.9 M | eval
-------------------------------------------
149 K     Trainable params
10.8 M    Non-trainable params
10.9 M    Total params
43.726    Total estimated model params size (MB)
0         Modules in train mode
339       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined